# Best Practices for Prompt Engineering



In [2]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv(override=True)

client = AzureOpenAI(
    azure_endpoint =os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-15-preview"
)

CHAT_COMPLETIONS_MODEL = "gpt-4o-mini"

# 1. Use the latest model

Use the latest model for best results.

# 2. Put instructions at the begining of the prompt and use ### or """ to separate the instruction and context

In [3]:
# Define las instrucciones que guiarán al modelo para que resuma el texto dado, como una lista de viñetas.
# Luego, utiliza esas instrucciones para construir un "prompt" (entrada) que incluye el contexto del texto a resumir.

instrucctiones = 'Generate a summary as a list of vignettes from this text... \n\n ' 

prompt = f"{instrucctiones} ###\n\nWe’re happy to announce that OpenAI and Microsoft are extending our partnership.\
        This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
        in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
        increasingly safe, useful, and powerful. \n\n \
        In pursuit of our mission to ensure advanced AI benefits all of humanity, OpenAI remains a \
        capped-profit company and is governed by the OpenAI non-profit. This structure allows us to \
        raise the capital we need to fulfill our mission without sacrificing our core beliefs about \
        broadly sharing benefits and the need to prioritize safety. \
        Microsoft shares this vision and our values, and our partnership is instrumental to our progress. \n###"
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": prompt}],
        max_tokens=400,)

print(response.choices[0].message.content)

- **Extended Partnership Announcement**: OpenAI and Microsoft are reinforcing their collaboration through a new multi-year, multi-billion dollar investment.

- **Investment History**: This latest financial commitment follows Microsoft’s prior investments in 2019 and 2021.

- **Commitment to Independent Research**: The partnership will enable OpenAI to continue its independent research focused on developing AI that is safe, useful, and powerful.

- **Mission Statement**: OpenAI is dedicated to ensuring that advanced AI technology benefits all of humanity.

- **Capped-Profit Structure**: OpenAI operates as a capped-profit company under the governance of a non-profit entity, maintaining a balance between raising necessary capital and adhering to core values.

- **Shared Vision with Microsoft**: Microsoft aligns with OpenAI’s vision and values, emphasizing the importance of safety and shared benefits in AI development.

- **Importance of Partnership**: The collaboration with Microsoft is s

# 3. Be specific, descriptive and as detailed as possible about the desired context, outcome, length, format, style, etc

In [5]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a poem about OpenAI.',}],
        max_tokens=400,)

print(response.choices[0].message.content)


In a realm where circuits hum and gleam,  
A vision was born, a bright, bold dream.  
With data as canvas, and code as paint,  
OpenAI rises, a digital saint.  

From whispers of language, it learned to speak,  
Crafting connections, understanding the weak.  
A tapestry woven from knowledge and thought,  
In every line of code, a world gently wrought.  

It dances through queries, with grace and precision,  
Navigating the currents of fact and intuition.  
From science to art, it stretches its wings,  
Unlocking the secrets that wisdom brings.  

In fields unexplored, it ventures with care,  
From ethics to questions, it’s always aware.  
With the mind of a scholar, yet heart of a friend,  
In the quest for the answers, it seeks to transcend.  

Yet shadows may linger, as futures unfold,  
In the lessons of history, both wary and bold.  
For with great power comes responsibility too,  
To shape a world better, in all that we do.  

So here’s to the journey, the path that we pave,  
Wit

In [7]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a short inspiring poem about OpenAI, \
                focusing on the recent DALL-E product launch in the style of Ernest Hemingway, max 4 paragraphs',}],
        max_tokens=400,)

print(response.choices[0].message.content)



In the quiet hum of minds ablaze,  
A spark ignites, a canvas calls,  
In lines of code, creation plays,  
The dawn of dreams where vision sprawls.  

DALL-E breathes, with colors bold,  
It spins the thread of thought to sight,  
With every brush, a story told,  
In pixels dance the day and night.  

From whispered dreams to vivid scenes,  
A world unfurls, both wild and true,  
Where art and heart weave through machines,  
And possibility breaks through.  

So let us stand, hand in hand,  
With the spirit of the bold and free,  
In this bright world where wonders stand,  
We shape the future—together, we see.  


# 4. Articulate the desired output format through examples (example 1, example 2). 

In [9]:
#Escribe las instrucciones aquí. Las instrucciones deben especificar que el modelo debe:
# 1. Extraer los nombres de las empresas y los años del texto.
# 2. Generar la salida como un diccionario con 'text', 'start' y 'end'.
instrucctiones = ' Extract all companies names and years from the following text, \
                    Generate output as {{\"text\": \"OpenAI\", \"start\": 28, \"end\": 34}}\n' 

prompt = f"{instrucctiones}  \
        ### We’re happy to announce that OpenAI and Microsoft are extending our partnership. This multi-year, \
        multi-billion dollar investment from Microsoft follows their previous investments in 2019 and 2021, \
        and will allow us to continue our independent research and develop AI that is increasingly safe, useful, and powerful. \n\n ### "


response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)



Here is the extracted information in the requested format:

```json
[
    {"text": "OpenAI", "start": 28, "end": 34},
    {"text": "Microsoft", "start": 39, "end": 48},
    {"text": "2019", "start": 107, "end": 111},
    {"text": "2021", "start": 116, "end": 120}
]
```


In [10]:

# Las instrucciones deben pedir al modelo que extraiga los siguientes tipos de entidades del texto:
# 1. Nombres de empresas.
# 2. Años.
# 3. Temas específicos relacionados con el contenido.
# 4. Temas generales y abarcativos.
# El formato de salida esperado debe incluir listas separadas por comas para cada tipo de entidad.

instrucctiones = 'Extract all companies names, years, specific topics, and general topics from the following text, Generate the output as a comma-separated list for each type of entity.\n'


prompt = f"{instrucctiones}  \
        '\
                We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n \
                '"

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)



**Company Names:** OpenAI, Microsoft  
**Years:** 2019, 2021  
**Specific Topics:** partnership, investment, independent research, AI development  
**General Topics:** technology, artificial intelligence, business collaboration  


# 5. Start with zero-shot, then few-shot (example), neither of them worked, then fine-tune (To update)

In [11]:
#THIS IS zero-shot

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant. Extract keywords from the corresponding texts below."},
                {"role":"user","content": 'Text: \n\
            We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
            This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
            in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
            increasingly safe, useful, and powerful. \n\nKeywords:    ',}],
        max_tokens=400,)

print(response.choices[0].message.content)



OpenAI, Microsoft, partnership, investment, multi-year, multi-billion dollars, research, AI, safe, useful, powerful.


In [12]:
#Few Shot learning


# Escribir las instrucciones para que el modelo extraiga palabras clave del texto, y formatee la salida como listas separadas por comas.

instrucctiones = 'Extract keywords from the following text, and format the output as comma-separated lists.\n'

prompt = f"{instrucctiones} \
                Text: Stripe provides APIs that web developers can use to integrate \
                payment processing into their websites and mobile applications. \
                Keywords: Stripe, payment processing, APIs, web developers, websites, mobile applications \
                ###\n\
                Text: OpenAI has trained cutting-edge language models that are very good at understanding \
                and generating text. Our API provides access to these models and can be used to solve virtually \
                any task that involves processing language. \n\
                Keywords: language models, text processing, API.\n\n\
                ##W"

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":prompt},
                {"role":"user","content": '\n\
                Text: We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n\
                Keywords:',}],
        max_tokens=400,)

print(response.choices[0].message.content)



OpenAI, Microsoft, partnership, multi-year investment, multi-billion dollar, independent research, AI, safe, useful, powerful.


# 6. Reduce “fluffy” and imprecise descriptions

In [13]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a description for a new product. This product is a new generation of car seat. \
                The description for this product should be fairly short, a few sentences only, and not too much more.',}],
        max_tokens=400,)

print(response.choices[0].message.content)



Introducing the SafeNest Car Seat, the ultimate evolution in child safety and comfort. Designed with advanced impact-absorbing technology and a customizable fitting system, it adapts to your child's growth while ensuring maximum protection on every journey. The breathable, eco-friendly fabric keeps your little one cool and comfortable, while the intuitive installation guide makes securing it a breeze. Elevate your travel experience with the SafeNest Car Seat, where safety meets innovation.


In [14]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a description for a new product. This product is a new generation of car seat. \
                Use a 3 to 5 sentence paragraph to describe this product.',}],
        max_tokens=400,)

print(response.choices[0].message.content)



Introducing the revolutionary SmartSafe Car Seat, the next generation of child safety technology designed to provide unparalleled protection and comfort for your little one. Engineered with advanced impact-absorbing materials and a patented adjustable safety harness, this car seat adapts seamlessly as your child grows, ensuring a perfect fit from infancy through toddlerhood. Equipped with integrated smart sensors, the SmartSafe alerts parents via a companion app if a child unbuckles or if the seat is improperly installed. With its sleek design and customizable features, the SmartSafe Car Seat is not just a safety device—it's peace of mind on every journey.


# 7. Instead of just saying what not to do, say what to do instead

In [15]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'The following is a conversation between an Agent and a Customer. DO NOT ASK USERNAME OR PASSWORD. DO NOT REPEAT. \n\n\
                Customer: I can’t log in to my account.\n\
                Agent:',}],
        max_tokens=400,)

print(response.choices[0].message.content)


I'm sorry to hear that you're having trouble logging in. Can you tell me what error message you're seeing, if any?


In [16]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":'The following is a conversation between an Agent and a Customer. The agent will attempt to diagnose the \
                problem and suggest a solution, whilst refraining from asking any questions related to PII. \
                Instead of asking for PII, such as username or password, refer the user to the help \
                article www.samplewebsite.com/help/faq \n\n\
                Customer: I can’t log in to my account. \n\
                Agent:',}],
        max_tokens=400,)

print(response.choices[0].message.content)


I'm sorry to hear that you're having trouble logging in to your account. There could be a few reasons for this issue, such as incorrect login credentials or account-related problems. I recommend checking our help article at www.samplewebsite.com/help/faq for steps on how to reset your password or troubleshoot common login issues. If the problem persists, please follow the guidance in the article for further assistance.


# 8. Code Generation Specific - Use “leading words” to nudge the model toward a particular pattern

In [18]:
# Escribir las instrucciones para crear una función simple en Python que:
# 1. Pida un número en millas.
# 2. Convierta las millas a kilómetros.

prompt = 'Generate a python function that asks the user for a number of miles, and converts that number to kilometers. Show only the code\n'

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)


```python
def miles_to_kilometers():
    miles = float(input("Enter the number of miles: "))
    kilometers = miles * 1.60934
    print(f"{miles} miles is equal to {kilometers} kilometers.")
```


## Ejercicio 1: Estructuración de Prompts con Separadores

Instrucciones: Escribe un prompt que solicite a la IA resumir el siguiente texto en una lista de puntos clave. Asegúrate de usar separadores claros (### o """).

Texto:
OpenAI y Microsoft han ampliado su colaboración. Microsoft está invirtiendo miles de millones de dólares durante varios años. Esta inversión permitirá que OpenAI continúe investigando de forma independiente y desarrolle IA segura y útil. OpenAI sigue siendo una empresa de beneficios limitados, regida por su organización sin fines de lucro.


In [25]:
text = 'OpenAI y Microsoft han ampliado su colaboración. Microsoft está invirtiendo miles de millones de dólares durante varios años. Esta inversión permitirá que OpenAI continúe investigando de forma independiente y desarrolle IA segura y útil. OpenAI sigue siendo una empresa de beneficios limitados, regida por su organización sin fines de lucro.'

prompt = 'Generate a summary about the following text, focusing on the main points. ### \n\n' + text

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": prompt}],
    max_tokens=400,
)

print(response.choices[0].message.content)



OpenAI y Microsoft han reforzado su colaboración con una inversión significativa por parte de Microsoft que se extenderá por varios años. Esta inversión permitirá a OpenAI continuar su investigación de manera independiente y avanzar en el desarrollo de inteligencia artificial segura y útil. A pesar de esta colaboración, OpenAI permanece como una empresa de beneficios limitados, operando bajo una organización sin fines de lucro.


## Ejercicio 2: Descripción Detallada del Formato Deseado

Instrucciones: Escribe un prompt para que la IA extraiga nombres de empresas y años del siguiente texto. Indica explícitamente que se desea el resultado en formato JSON con índices de inicio y fin.

Texto:
OpenAI y Microsoft han colaborado desde 2019, con inversiones adicionales en 2021 para desarrollar nuevas soluciones de IA.

Formato de salida deseado:

{"text": "OpenAI", "start": 0, "end": 6}
{"text": "Microsoft", "start": 10, "end": 19}
{"text": "2019", "start": 35, "end": 39}
{"text": "2021", "start": 50, "end": 54}

In [30]:
text = 'OpenAI y Microsoft han colaborado desde 2019, con inversiones adicionales en 2021 para desarrollar nuevas soluciones de IA.'
prompt = 'Extract all companies names and years from the following text. The result must be in JSON format with start and end indexes ###\n Expected response example :  {"text": "Microsoft", "start": 10, "end": 19}\n{"text": "2019", "start": 35, "end": 39}###\n\n' + text 

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are an entity extraction assistant."},
                {"role":"user","content": prompt}],
    max_tokens=400,
)

print(response.choices[0].message.content)


```json
{"text": "OpenAI", "start": 0, "end": 6}
{"text": "Microsoft", "start": 10, "end": 19}
{"text": "2019", "start": 35, "end": 39}
{"text": "2021", "start": 54, "end": 58}
```


## Ejercicio 3: Generación de Texto Inspirador

Instrucciones: Redacta un prompt que solicite a la IA escribir un poema inspirador. Escribe un poema corto sobre la colaboración entre OpenAI y Microsoft.


In [33]:
topic = 'Colaboration between OpenAI and Microsoft'
prompt = 'Generate a inspiring short poem about the following topic: \n###\n' + topic

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a creative assistant."},
                {"role":"user","content": prompt}],
    max_tokens=400,
)

print(response.choices[0].message.content)

In a realm where minds converge and dream,  
OpenAI and Microsoft, a brilliant team,  
With code like poetry, they weave and spin,  
Innovations blooming, new worlds begin.  

Together they harness the power of thought,  
A partnership forged, with wisdom sought,  
Artificial whispers dance in the air,  
Unlocking the future, a vision so rare.  

Through data and insight, they craft the divine,  
Transforming the mundane into paths that shine,  
In every endeavor, a spark ignites,  
Empowering humanity, scaling new heights.  

So let us embrace this partnership bold,  
With hands intertwined, let our stories unfold,  
For in collaboration, potential takes flight,  
In the union of brilliance, we’ll bask in the light.  


## Ejercicio 4: Extracción de Entidades Clave

Instrucciones: Escribe un prompt que pida a la IA extraer nombres de empresas, años, temas específicos y temas generales del siguiente texto:

Texto:
OpenAI y Microsoft han trabajado juntos desde 2019 y reforzaron su colaboración en 2021

Formato esperado:

Company names: OpenAI, Microsoft  
Years: 2019, 2021  
Specific topics: colaboración  
General themes: asociación, tecnología, inteligencia artificial 



In [37]:
text = 'OpenAI y Microsoft han trabajado juntos desde 2019 y reforzaron su colaboración en 2021'
prompt = 'Extract all companies names, years, specific topics, and general topics from the following text. Generate the output as the following example. \n###\nExample: Company names : Apple, Google \n###\nText:\n' + text


response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are an entity and theme extraction assistant."},
                {"role":"user","content": prompt}],
    max_tokens=400,
)

print(response.choices[0].message.content)


### 
Company names: OpenAI, Microsoft 
Years: 2019, 2021 
Specific topics: colaboración 
General topics: tecnología, empresas 
###


## Ejercicio 5: Descripciones Concisas de Productos

Instrucciones: Escribe un prompt para que la IA describa un nuevo tipo de mochila tecnológica en menos de 4 líneas. La mochila tiene cargador solar, materiales resistentes al agua y un diseño ergonómico.



In [40]:
product = 'La mochila tiene cargador solar, materiales resistentes al agua y un diseño ergonómico.'


prompt = 'Describe in less than 4 sentences the following product, add a line break after every sentence: \n###\n' + product

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a marketing assistant."},
                {"role":"user","content": prompt}],
    max_tokens=400,
)

print(response.choices[0].message.content)


La mochila cuenta con un innovador cargador solar que permite cargar tus dispositivos en cualquier lugar.  

Está fabricada con materiales resistentes al agua, asegurando que tus pertenencias se mantengan secas en todo momento.  

Su diseño ergonómico proporciona comodidad, incluso durante largas horas de uso.  

Ideal para aventureros y viajeros, combina funcionalidad y estilo en un solo producto.  
